## Module Import

In [19]:
# self defined Modules
from myModules.utils import merge
from myModules.plot.plotNetwork import vocaDict, Network

# General Modules
import pandas as pd
import numpy as np
import warnings
import pickle
from itertools import product
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

# visualize
import seaborn as sns
import matplotlib.pyplot as plt

# NLP
from sklearn.feature_extraction.text import TfidfVectorizer

## Data Load

In [2]:
DATA_ROOT = './processed-data/'

PERIOD_1 = DATA_ROOT + 'period-1/'
PERIOD_2 = DATA_ROOT + 'period-2/'
PERIOD_3 = DATA_ROOT + 'period-3/'

RESULT_ROOT = './Result/3구간/'

RESULT_1 = RESULT_ROOT + '/1시기/ST/'
RESULT_2 = RESULT_ROOT + '/2시기/ST/'
RESULT_3 = RESULT_ROOT + '/3시기/ST/'

In [3]:
with open(PERIOD_1+"lemmatized-all.pkl", "rb") as f:
    all_1 = pickle.load(f)
with open(PERIOD_1+"lemmatized-noun.pkl", "rb") as f:
    noun_1 = pickle.load(f)
with open(PERIOD_1+"lemmatized-verb.pkl", "rb") as f:
    verb_1 = pickle.load(f)
with open(PERIOD_1+"lemmatized-adjective.pkl", "rb") as f:
    adjective_1 = pickle.load(f)
with open(PERIOD_1+"lemmatized-adverb.pkl", "rb") as f:
    adverb_1 = pickle.load(f)


with open(PERIOD_2+"lemmatized-all.pkl", "rb") as f:
    all_2 = pickle.load(f)
with open(PERIOD_2+"lemmatized-noun.pkl", "rb") as f:
    noun_2 = pickle.load(f)
with open(PERIOD_2+"lemmatized-verb.pkl", "rb") as f:
    verb_2 = pickle.load(f)
with open(PERIOD_2+"lemmatized-adjective.pkl", "rb") as f:
    adjective_2 = pickle.load(f)
with open(PERIOD_2+"lemmatized-adverb.pkl", "rb") as f:
    adverb_2 = pickle.load(f)

with open(PERIOD_3+"lemmatized-all.pkl", "rb") as f:
    all_3 = pickle.load(f)
with open(PERIOD_3+"lemmatized-noun.pkl", "rb") as f:
    noun_3 = pickle.load(f)
with open(PERIOD_3+"lemmatized-verb.pkl", "rb") as f:
    verb_3 = pickle.load(f)
with open(PERIOD_3+"lemmatized-adjective.pkl", "rb") as f:
    adjective_3 = pickle.load(f)
with open(PERIOD_3+"lemmatized-adverb.pkl", "rb") as f:
    adverb_3 = pickle.load(f)

## Phi Coefficient

In [4]:
def tokens_into_one_string(data):
    result = []

    for article in data:
        result.append(' '.join(article))
    
    return result

In [5]:
corpus = tokens_into_one_string(all_1)

In [6]:
tfidf_vectorizer = TfidfVectorizer().fit(corpus)

In [7]:
mat = tfidf_vectorizer.transform(corpus).toarray()

In [8]:
df = pd.DataFrame(mat)

In [10]:
corr_df = df.corr()

In [15]:
idx_list = list(range(len(corr_df)))

In [88]:
edges = []
corr_thresh = 0.8
length = len(corr_df)

for row in tqdm(range(length)):
    for col in range(row + 1, length):
        if (corr_df.loc[row, col] >= corr_thresh):
            edges.append([row, col, corr_df.loc[row, col]])

  0%|          | 0/6707 [00:00<?, ?it/s]

In [89]:
edge_df = pd.DataFrame(edges)

In [90]:
edge_df.columns = ['word1', 'word2', 'corr']

In [91]:
sorted_edge_df = edge_df.sort_values(by='corr').reset_index().drop(['index'], axis=1)

In [112]:
sorted_edge_df[-5001:-1]

,word1,word2,corr
1440563,3551,5618,1.0
1440564,3963,6137,1.0
1440565,3963,6122,1.0
1440566,4312,5183,1.0
1440567,1294,6138,1.0
...,...,...,...
1445558,1378,1912,1.0
1445559,1378,1913,1.0
1445560,1378,1930,1.0
1445561,1378,1932,1.0


In [54]:
voca_dict = tfidf_vectorizer.vocabulary_

In [80]:
def get_token_from_id(id, voca_dict):
    for key, val in voca_dict.items():
        if val == id:
            return key

def edge_df_with_token(edge_df, voca_dict):
    result = []

    for idx in tqdm(range(len(edge_df))):
        result.append([get_token_from_id(edge_df.loc[idx, 'word1'], voca_dict), get_token_from_id(edge_df.loc[idx, 'word2'], voca_dict), edge_df.loc[idx, 'corr']])
        print(result)
    
    result = pd.DataFrame(result)
    result.columns = ['word1', 'word2', 'corr']

    return result

In [117]:
get_token_from_id(1294, voca_dict)

'copper'

In [118]:
get_token_from_id(6138, voca_dict)

'transcontinental'